In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math
import random

PI = 3.141592 

In [2]:
def configuracoes():

  global program, window

  glfw.init()
  glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
  window = glfw.create_window(700, 700, "Esfera", None, None)
  glfw.make_context_current(window)

  vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """
  
  fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """
  
  # Request a program and shader slots from GPU
  program  = glCreateProgram()
  vertex   = glCreateShader(GL_VERTEX_SHADER)
  fragment = glCreateShader(GL_FRAGMENT_SHADER)

  # Set shaders source
  glShaderSource(vertex, vertex_code)
  glShaderSource(fragment, fragment_code)

  # Compile shaders
  glCompileShader(vertex)
  if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")
  
  glCompileShader(fragment)
  if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

  # Attach shader objects to the program
  glAttachShader(program, vertex)
  glAttachShader(program, fragment)

  # Build program
  glLinkProgram(program)
  if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
  # Make program the default program
  glUseProgram(program)

In [3]:
# Entrada: angulo de t, altura h, raio r
# Saida: coordenadas no cilindro
def CoordCilindro(t, h, r):
  x = r * math.cos(t)
  y = r * math.sin(t)
  z = h
  return (x,y,z)


def CIRILO(h,r):

  num_sectors = 20 # qtd de sectors (longitude)
  num_stacks = 20 # qtd de stacks (latitude)

  # grid sectos vs stacks (longitude vs latitude)
  sector_step = (PI*2)/num_sectors # variar de 0 até 2π
  stack_step = h/num_stacks # variar de 0 até H

  # vamos gerar um conjunto de vertices representantes poligonos
  # para a superficie da esfera.
  # cada poligono eh representado por dois triangulos
  vertices_list = []
  for j in range(0,num_stacks): # para cada stack (latitude)
      
    for i in range(0,num_sectors): # para cada sector (longitude) 
        
      u = i * sector_step # angulo setor
      v = j * stack_step # altura da stack
      
      un = 0 # angulo do proximo sector
      if i+1==num_sectors:
          un = PI*2
      else: un = (i+1)*sector_step
          
      vn = 0 # altura da proxima stack
      if j+1==num_stacks:
          vn = h
      else: vn = (j+1)*stack_step
      
      # verticies do poligono
      p0=CoordCilindro(u, v, r)
      p1=CoordCilindro(u, vn, r)
      p2=CoordCilindro(un, v, r)
      p3=CoordCilindro(un, vn, r)
      
      # triangulo 1 (primeira parte do poligono)
      vertices_list.append(p0)
      vertices_list.append(p2)
      vertices_list.append(p1)
      
      # triangulo 2 (segunda e ultima parte do poligono)
      vertices_list.append(p3)
      vertices_list.append(p1)
      vertices_list.append(p2)
      
      if v == 0:
          vertices_list.append(p0)
          vertices_list.append(p2)
          vertices_list.append(CoordCilindro(0, v, 0))
      if vn == h:
          #faz um triangulo a partir do mesmo angulo u, mas com as alturas em h = vn
          vertices_list.append(p1)
          vertices_list.append(p3)
          vertices_list.append(CoordCilindro(0, vn, 0))

  total_vertices = len(vertices_list)
  vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
  vertices['position'] = np.array(vertices_list)

  return vertices


In [4]:
def passar_para_gpu(vertices):
  # Request a buffer slot from GPU
  buffer = glGenBuffers(1)
  # Make this buffer the default one
  glBindBuffer(GL_ARRAY_BUFFER, buffer)


  # Upload data
  glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
  glBindBuffer(GL_ARRAY_BUFFER, buffer)

  # Bind the position attribute
  # --------------------------------------
  stride = vertices.strides[0]
  offset = ctypes.c_void_p(0)

  loc = glGetAttribLocation(program, "position")
  glEnableVertexAttribArray(loc)
  glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)
  

In [5]:
configuracoes()

vertices = CIRILO(0.9, 0.1)

passar_para_gpu(vertices)

DAQUI PRA BAIXO AINDA N MODULARIZEI.


A CALL É: ENTENDER, JUNTO DO GPT E DO TRAB DO MERLIN, QUAIS SAO OS PACOTES DE CODIGO, QUE PODEMOS SEPARAR E MODULARIZAR.
NO LOOP DE BAIXO EU SEI Q DEVE HAVER UM "DRAW_TAL_COISA", MAS N SEI ONDE NEM COMO AINDA

In [ ]:
# Não sei oq exatamente isso faz, mas no notebook do professor, 
# tá falando q pega a localização da variável de cor (uniform) do Fragment Shader
# pra poder alterar ela no laço da janela. N sei oq significa.

loc_color = glGetUniformLocation(program, "color")
glfw.show_window(window)

In [8]:

#ângulo de rotação do objeto. Inicialmente é zero, mas vai sendo decrementada 
# a cada iteração para gerar o movimento de rotação.
d = 0.0

#Isso garante que o OpenGL renderize objetos 3D corretamente, levando em consideração 
#a profundidade dos vértices e ocultando superfícies que deveriam estar atrás de outras.
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() #Leitura de eventos da janela
    
    ### apenas para visualizarmos o cilindro rotacionando
    d -= 0.01 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    # Limpa a tela para preparar o próximo quadro.
    # O fundo é definido como branco (1.0, 1.0, 1.0, 1.0).
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_d,  0.0, sin_d, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_d, 0.0, cos_d, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_x)
    mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)

    # a matriz de transformação é passada para o shader, que aplicará 
    # essa rotação a todos os vértices do objeto.
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
    for triangle in range(0,len(vertices),3):
       
        random.seed( triangle )
        R = random.random()
        G = random.random()
        B = random.random()  

        # Essa parte aqui faz o desenho 
        glUniform4f(loc_color, R, G, B, 1.0)
        glDrawArrays(GL_TRIANGLES, triangle, 3)     


    
    glfw.swap_buffers(window)

glfw.terminate()